In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

sys.path.append('/home/peter/Code/deepomics')
from neuralnetwork import NeuralNet, NeuralTrainer
import train as fit 
import visualize, utils

from lasagne import layers, nonlinearities, updates, objectives, init 
from lasagne.layers import get_output, get_output_shape, get_all_params
import theano.tensor as T
import theano
np.random.seed(247) # for reproducibility


Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, cuDNN 5105)


In [2]:
filename = 'processed_dataset.hdf5'
data_path = '/home/peter/Code/tensorflow/data'
file_path = os.path.join(data_path, filename)
group_name = ['processed_data']
dataset = h5py.File(file_path,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

train = (dtf.transpose([0,3,1,2]), ltf)
valid = (dtf_crossval.transpose([0,3,1,2]), ltf_crossval)


CPU times: user 324 ms, sys: 588 ms, total: 912 ms
Wall time: 911 ms


# original model

In [3]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')

    net['conv2'] = layers.Conv2DLayer(net['conv1'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2'], pool_size=(1, 10), stride=(1, 10)) # 25

    net['conv3'] = layers.Conv2DLayer(net['conv2_pool'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')

    net['dense1'] = layers.DenseLayer(net['conv3'], num_units=100, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=nonlinearities.rectify)
    
    net['output'] = layers.DenseLayer(net['dense1'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


In [4]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.40137 -- accuracy=83.96%  
  valid loss:		0.37519
  valid accuracy:	0.85309+/-0.00006
  valid auc-roc:	0.90830+/-0.00002
  valid auc-pr:		0.89853+/-0.00614
saving model parameters to: ../results/test/original_model_best.pickle
Epoch 2 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.37267 -- accuracy=85.35%  
  valid loss:		0.36890
  valid accuracy:	0.85491+/-0.00003
  valid auc-roc:	0.91288+/-0.00002
  valid auc-pr:		0.90612+/-0.00803
saving model parameters to: ../results/test/original_model_best.pickle
Epoch 3 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.36676 -- accuracy=85.54%  
  valid loss:		0.36356
  valid accuracy:	0.85665+/-0.00001
  valid auc-roc:	0.91432+/-0.00001
  valid auc-pr:		0.90777+/-0.00744
saving model parameters to: ../results/test/original_model_best.pickle
Epoch 4 out of 500 
[==============================] 100.0% -- time=0s -- lo

<neuralnetwork.NeuralTrainer instance at 0x7f26ef154998>

In [5]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.35232
  test  accuracy:	0.86048+/-0.00003
  test  auc-roc:	0.91995+/-0.00000
  test  auc-pr:		0.91446+/-0.00609


0.35231954771865853

# original model w/ batch norm and dropout and relu

In [6]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_active'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_active'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.NonlinearityLayer(net['conv3_norm'], nonlinearity=nonlinearities.rectify)
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=100, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1'], nonlinearity=nonlinearities.rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model_bells'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


In [7]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.39669 -- accuracy=84.36%  
  valid loss:		0.36735
  valid accuracy:	0.85635+/-0.00004
  valid auc-roc:	0.91488+/-0.00000
  valid auc-pr:		0.90728+/-0.00652
saving model parameters to: ../results/test/original_model_bells_best.pickle
Epoch 2 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.36848 -- accuracy=85.59%  
  valid loss:		0.35560
  valid accuracy:	0.85806+/-0.00003
  valid auc-roc:	0.91884+/-0.00002
  valid auc-pr:		0.91258+/-0.00579
saving model parameters to: ../results/test/original_model_bells_best.pickle
Epoch 3 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.36197 -- accuracy=85.82%  
  valid loss:		0.35319
  valid accuracy:	0.86026+/-0.00001
  valid auc-roc:	0.92048+/-0.00000
  valid auc-pr:		0.91454+/-0.00537
saving model parameters to: ../results/test/original_model_bells_best.pickle
Epoch 4 out of 500 
[==============================] 100.0

<neuralnetwork.NeuralTrainer instance at 0x7f26ef154950>

In [8]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.33226
  test  accuracy:	0.86687+/-0.00000
  test  auc-roc:	0.93043+/-0.00000
  test  auc-pr:		0.92683+/-0.00208


0.33226152080699856

# modified network (wider + batch norm + dropout + prelu)

In [9]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.ParametricRectifierLayer(net['conv1_norm'], alpha=init.Constant(0.25))
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_active'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.ParametricRectifierLayer(net['conv2_norm'], alpha=init.Constant(0.25))
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_active'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=128, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.ParametricRectifierLayer(net['conv3_norm'], alpha=init.Constant(0.25))
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=256, W=init.HeNormal(), 
                                     b=init.Constant(0.1), nonlinearity=None)
    net['dense1_active'] = layers.ParametricRectifierLayer(net['dense1_norm'], alpha=init.Constant(0.25))
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model_bells_whistles'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

making directory: ../results/test/original_model_bells_whistles
compiling model


In [10]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=10, patience=0, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=10, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=5, patience=0, verbose=1)



Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.37656 -- accuracy=85.38%  
  valid loss:		0.35316
  valid accuracy:	0.86072+/-0.00000
  valid auc-roc:	0.92105+/-0.00000
  valid auc-pr:		0.91483+/-0.00416
saving model parameters to: ../results/test/original_model_bells_whistles_best.pickle
Epoch 2 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.36055 -- accuracy=86.04%  
  valid loss:		0.34587
  valid accuracy:	0.86301+/-0.00000
  valid auc-roc:	0.92435+/-0.00000
  valid auc-pr:		0.91946+/-0.00375
saving model parameters to: ../results/test/original_model_bells_whistles_best.pickle
Epoch 3 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.35643 -- accuracy=86.19%  
  valid loss:		0.34308
  valid accuracy:	0.86351+/-0.00000
  valid auc-roc:	0.92540+/-0.00000
  valid auc-pr:		0.92085+/-0.00323
saving model parameters to: ../results/test/original_model_bells_whistles_best.pickle
Epoch 4 out of 500 
[==========

KeyboardInterrupt: 

In [11]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.32901
  test  accuracy:	0.86730+/-0.00000
  test  auc-roc:	0.93208+/-0.00000
  test  auc-pr:		0.92913+/-0.00068


0.32900719022581171

# normal residual model

In [60]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(3,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_2_resid'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.NonlinearityLayer(net['conv3_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=100, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.1)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = 'original_model_residual'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=10, patience=0, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=10, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=5, patience=0, verbose=1)


In [66]:
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=4000, num_epochs=5, patience=5, verbose=1)

Epoch 1 out of 5 
[==============================] 100.0% -- time=0s -- loss=0.32940 -- accuracy=86.98%  
  valid loss:		0.32311
  valid accuracy:	0.86969+/-0.00000
  valid auc-roc:	0.93522+/-0.00000
  valid auc-pr:		0.93244+/-0.00028
saving model parameters to: ../results/test/original_model_residual_best.pickle
Epoch 2 out of 5 
[==============================] 100.0% -- time=0s -- loss=0.32876 -- accuracy=86.99%  
  valid loss:		0.32284
  valid accuracy:	0.86973+/-0.00000
  valid auc-roc:	0.93542+/-0.00000
  valid auc-pr:		0.93269+/-0.00051
saving model parameters to: ../results/test/original_model_residual_best.pickle
Epoch 3 out of 5 
[==============================] 100.0% -- time=0s -- loss=0.32896 -- accuracy=86.96%  
  valid loss:		0.32279
  valid accuracy:	0.86928+/-0.00000
  valid auc-roc:	0.93541+/-0.00000
  valid auc-pr:		0.93269+/-0.00033
saving model parameters to: ../results/test/original_model_residual_best.pickle
Epoch 4 out of 5 
[==============================] 100.

KeyboardInterrupt: 

In [67]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.32277
  test  accuracy:	0.86929+/-0.00000
  test  auc-roc:	0.93541+/-0.00000
  test  auc-pr:		0.93271+/-0.00030


0.32277447251302466

# residual model (modified)

In [70]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.ParametricRectifierLayer(net[name+'_1resid_norm'], alpha=init.Constant(0.25))
            
        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.ParametricRectifierLayer(net[name+'_residual'], alpha=init.Constant(0.25))

        return net
    
    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.ParametricRectifierLayer(net[name+'_1resid_norm'], alpha=init.Constant(0.25))

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.ParametricRectifierLayer(net[name+'_residual'], alpha=init.Constant(0.25))

        return net

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(2, 5), stride=(1, 1),    # 325
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.ParametricRectifierLayer(net['conv1_norm'], alpha=init.Constant(0.25))
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(3, 5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_2_resid'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(2, 5), stride=(1, 1), # 60
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.ParametricRectifierLayer(net['conv2_norm'], alpha=init.Constant(0.25))
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1, 5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 10), stride=(1, 10)) # 12
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=128, filter_size=(1, 1), stride=(1, 1), # 9
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.ParametricRectifierLayer(net['conv3_norm'], alpha=init.Constant(0.25))
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)
    
    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=256, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.ParametricRectifierLayer(net['dense1_norm'], alpha=init.Constant(0.25))
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.1)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.3)
        
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
        
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.0003, 
                    "l2": 1e-5
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = '1D_model_resid'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


In [71]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=10, patience=0, verbose=1)

# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=10, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=5, patience=0, verbose=1)


Epoch 1 out of 10 
[==============================] 100.0% -- time=0s -- loss=0.40272 -- accuracy=85.20%  
  valid loss:		0.35967
  valid accuracy:	0.85969+/-0.00002
  valid auc-roc:	0.91910+/-0.00002
  valid auc-pr:		0.91246+/-0.00461
saving model parameters to: ../results/test/1D_model_resid_best.pickle
Epoch 2 out of 10 
[==============================] 100.0% -- time=0s -- loss=0.37424 -- accuracy=85.99%  
  valid loss:		0.35034
  valid accuracy:	0.86116+/-0.00002
  valid auc-roc:	0.92356+/-0.00000
  valid auc-pr:		0.91792+/-0.00366
saving model parameters to: ../results/test/1D_model_resid_best.pickle
Epoch 3 out of 10 
[==============================] 100.0% -- time=0s -- loss=0.36537 -- accuracy=86.20%  
  valid loss:		0.34944
  valid accuracy:	0.86300+/-0.00000
  valid auc-roc:	0.92505+/-0.00000
  valid auc-pr:		0.92028+/-0.00274
saving model parameters to: ../results/test/1D_model_resid_best.pickle
Epoch 4 out of 10 
[==============================] 100.0% -- time=0s -- loss=0

MemoryError: Error allocating 247296000 bytes of device memory (out of memory).
Apply node that caused the error: GpuAllocEmpty(Shape_i{0}.0, Shape_i{0}.0, TensorConstant{1}, Elemwise{Composite{(((i0 + (i1 * (Composite{(((i0 - i1) * i2) + i1)}(i2, i3, i4) // i1))) - Composite{(((i0 - i1) * i2) + i1)}(i2, i3, i4)) + i3)}}.0)
Toposort index: 220
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), ()]
Inputs strides: [(), (), (), ()]
Inputs values: [array(3000), array(64), array(1), array(322)]
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='half', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0})]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'op

In [72]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.31627
  test  accuracy:	0.87196+/-0.00000
  test  auc-roc:	0.93919+/-0.00000
  test  auc-pr:		0.93689+/-0.00210


0.31626673209154132

# 1D models

# 1D architecture

In [73]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    
    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(3, 11), stride=(1, 1),    # 320
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_2_resid'], pool_size=(1, 32), stride=(1, 32)) # 13
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_pool'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv1_dropout'], num_units=128, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.1)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = '1d_version'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


INFO (theano.gof.compilelock): Refreshing lock /home/peter/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /home/peter/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


In [76]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=10, patience=0, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=10, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=5, patience=0, verbose=1)


Epoch 1 out of 10 
[==============================] 100.0% -- time=0s -- loss=0.34254 -- accuracy=86.56%  
  valid loss:		0.33890
  valid accuracy:	0.86542+/-0.00000
  valid auc-roc:	0.92866+/-0.00000
  valid auc-pr:		0.92491+/-0.00224
saving model parameters to: ../results/test/1d_version_best.pickle
Epoch 2 out of 10 
[==============================] 100.0% -- time=0s -- loss=0.34299 -- accuracy=86.58%  
  valid loss:		0.33836
  valid accuracy:	0.86514+/-0.00000
  valid auc-roc:	0.92846+/-0.00000
  valid auc-pr:		0.92417+/-0.00267
saving model parameters to: ../results/test/1d_version_best.pickle
Epoch 3 out of 10 
[==============================] 100.0% -- time=0s -- loss=0.34332 -- accuracy=86.57%  
  valid loss:		0.33682
  valid accuracy:	0.86581+/-0.00000
  valid auc-roc:	0.92914+/-0.00000
  valid auc-pr:		0.92489+/-0.00272
saving model parameters to: ../results/test/1d_version_best.pickle
Epoch 4 out of 10 
[==============================] 100.0% -- time=0s -- loss=0.34290 -- ac

<neuralnetwork.NeuralTrainer instance at 0x7f26dd99def0>

In [77]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

  test  loss:		0.32840
  test  accuracy:	0.86813+/-0.00000
  test  auc-roc:	0.93255+/-0.00000
  test  auc-pr:		0.92928+/-0.00052


0.32840126553278232

# 1D residual model (modified)

In [ ]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    
    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net


    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape) # 330
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(3, 7), stride=(1, 1), # 324
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_2_resid'], pool_size=(1, 12), stride=(1, 12)) # 27
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_pool'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(1, 7), stride=(1, 1), # 21
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 7), stride=(1, 7)) # 3
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv2_dropout'], num_units=128, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.1)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.3
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory('../results', 'test')
output_name = '1D_model_resid'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=10, patience=0, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=10, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=5, patience=0, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=5, patience=0, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")